In [17]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

In [18]:
es.indices.refresh(index='posts')
es.cat.count(index='posts', params={"format": "json"})

/tmp/ipykernel_35375/2028796498.py:2: DeprecationWarning: The 'params' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.cat.count(index='posts', params={"format": "json"})


ListApiResponse([{'epoch': '1685421462', 'timestamp': '04:37:42', 'count': '78'}])